In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from scipy.stats import linregress
from pprint import pprint
import json
from datetime import datetime 

# Output unemployment (CSV)
ui_csv = pd.read_csv("Resources/DOL UI 2015 to 2020 6 States.csv")

# Assign Variable to Weeknum of Date column
ui_csv[["Filed week ended", "Reflecting Week Ended"]]=ui_csv[["Filed week ended", "Reflecting Week Ended"]].apply(pd.to_datetime)

# Assign Variable to Weeknum of Date column
weeknum = ui_csv['Reflecting Week Ended'].dt.isocalendar().week

# Add Weeknum column to Data / ISO week date standard (ISO-8601) / start Mon to end Sun
ui_csv['Weeknum'] = weeknum

# output csv file
ui_csv.to_csv("Resources/ui_clean.csv", index=False)
ui_csv.head()

,State,Filed week ended,Initial Claims,Reflecting Week Ended,Continued Claims,Covered Employment,Insured Unemployment Rate,Weeknum
0,California,2015-01-03,"40,784",2014-12-27,"420,154","15,245,478",2.76,52
1,California,2015-01-10,"66,906",2015-01-03,"423,923","15,354,250",2.76,1
2,California,2015-01-17,"62,191",2015-01-10,"464,163","15,354,250",3.02,2
3,California,2015-01-24,"40,605",2015-01-17,"390,977","15,354,250",2.55,3
4,California,2015-01-31,"58,673",2015-01-24,"468,557","15,354,250",3.05,4


In [2]:
# Output state abbreviation conversion file (CSV)
state_abbrevs = pd.read_csv("Resources/state_abbrev.csv")
state_abbrevs.tail()

,State,Abbrev,Code
46,Virginia,Va.,VA
47,Washington,Wash.,WA
48,West Virginia,W.Va.,WV
49,Wisconsin,Wis.,WI
50,Wyoming,Wyo.,WY


In [3]:
# Output unemployment (CSV)
ad_csv = pd.read_csv("Resources/CDC Anxiety Depression Frequency Last_7_Days.csv")

# Date format delimiting date column
ad_csv["Week Label"] = ad_csv["Week Label"].str.split(" -", n = 1, expand = True)  # Formatting Dates: Delimitting date after "-"
ad_csv.head()

,Indicator,Group,State,Subgroup,Week,Week Label,Value,Low CI,High CI,Confidence Interval,Quartile range
0,Symptoms of Depressive Disorder,National Estimate,United States,United States,1,Apr 23,23.5,22.7,24.3,22.7 - 24.3,NaN
1,Symptoms of Depressive Disorder,By Age,United States,18 - 29 years,1,Apr 23,32.7,30.2,35.2,30.2 - 35.2,NaN
2,Symptoms of Depressive Disorder,By Age,United States,30 - 39 years,1,Apr 23,25.7,24.1,27.3,24.1 - 27.3,NaN
3,Symptoms of Depressive Disorder,By Age,United States,40 - 49 years,1,Apr 23,24.8,23.3,26.2,23.3 - 26.2,NaN
4,Symptoms of Depressive Disorder,By Age,United States,50 - 59 years,1,Apr 23,23.2,21.5,25.0,21.5 - 25.0,NaN


In [4]:
# Issue: Week label col included (abbrev & full month name); Fix: replace the abbeviated values 
ad_csv["Week Label"] = ad_csv["Week Label"].replace(['Apr 23'],'April 23')

# # Add Year to Week Label
ad_csv["Week Label"] = (ad_csv["Week Label"] + ", 2020") ######## WARNING: if rerun this code more than once -adds a new 2020

# format "Week Label" as date
ad_csv['Week Label'] = [datetime.strptime(x, '%B %d, %Y') for x in ad_csv['Week Label']]
ad_csv

,Indicator,Group,State,Subgroup,Week,Week Label,Value,Low CI,High CI,Confidence Interval,Quartile range
0,Symptoms of Depressive Disorder,National Estimate,United States,United States,1,2020-04-23,23.5,22.7,24.3,22.7 - 24.3,NaN
1,Symptoms of Depressive Disorder,By Age,United States,18 - 29 years,1,2020-04-23,32.7,30.2,35.2,30.2 - 35.2,NaN
2,Symptoms of Depressive Disorder,By Age,United States,30 - 39 years,1,2020-04-23,25.7,24.1,27.3,24.1 - 27.3,NaN
3,Symptoms of Depressive Disorder,By Age,United States,40 - 49 years,1,2020-04-23,24.8,23.3,26.2,23.3 - 26.2,NaN
4,Symptoms of Depressive Disorder,By Age,United States,50 - 59 years,1,2020-04-23,23.2,21.5,25.0,21.5 - 25.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2515,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Virginia,Virginia,12,2020-07-16,38.3,34.0,42.7,34.0 - 42.7,37.2-40.5
2516,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Washington,Washington,12,2020-07-16,43.8,39.3,48.5,39.3 - 48.5,43.6-50.2
2517,Symptoms of Anxiety Disorder or Depressive Dis...,By State,West Virginia,West Virginia,12,2020-07-16,37.0,31.4,42.8,31.4 - 42.8,31.8-37.1
2518,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Wisconsin,Wisconsin,12,2020-07-16,33.3,28.9,37.9,28.9 - 37.9,31.8-37.1


In [5]:
# Assign Variable to Weeknum of Date column
weeknum = ad_csv['Week Label'].dt.isocalendar().week

# Add Weeknum column to Data / ISO week date standard (ISO-8601) / start Mon to end Sun
ad_csv['Weeknum']= weeknum

# output csv file
ad_csv.to_csv("Resources/ad_clean.csv", index=False)
ad_csv

,Indicator,Group,State,Subgroup,Week,Week Label,Value,Low CI,High CI,Confidence Interval,Quartile range,Weeknum
0,Symptoms of Depressive Disorder,National Estimate,United States,United States,1,2020-04-23,23.5,22.7,24.3,22.7 - 24.3,NaN,17
1,Symptoms of Depressive Disorder,By Age,United States,18 - 29 years,1,2020-04-23,32.7,30.2,35.2,30.2 - 35.2,NaN,17
2,Symptoms of Depressive Disorder,By Age,United States,30 - 39 years,1,2020-04-23,25.7,24.1,27.3,24.1 - 27.3,NaN,17
3,Symptoms of Depressive Disorder,By Age,United States,40 - 49 years,1,2020-04-23,24.8,23.3,26.2,23.3 - 26.2,NaN,17
4,Symptoms of Depressive Disorder,By Age,United States,50 - 59 years,1,2020-04-23,23.2,21.5,25.0,21.5 - 25.0,NaN,17
...,...,...,...,...,...,...,...,...,...,...,...,...
2515,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Virginia,Virginia,12,2020-07-16,38.3,34.0,42.7,34.0 - 42.7,37.2-40.5,29
2516,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Washington,Washington,12,2020-07-16,43.8,39.3,48.5,39.3 - 48.5,43.6-50.2,29
2517,Symptoms of Anxiety Disorder or Depressive Dis...,By State,West Virginia,West Virginia,12,2020-07-16,37.0,31.4,42.8,31.4 - 42.8,31.8-37.1,29
2518,Symptoms of Anxiety Disorder or Depressive Dis...,By State,Wisconsin,Wisconsin,12,2020-07-16,33.3,28.9,37.9,28.9 - 37.9,31.8-37.1,29
